# AutoGen - AgentChat 

Using AgentChat, we can quickly build agentic applications using preset agents.

In this notebook, let us start buildiong a single agent which uses tools. 

## Loading Environment variables

In [1]:
from dotenv import load_dotenv
import os 

_ = load_dotenv()

## Installation of `agentchat` and `ext[openai]` library

In [2]:
pip install -U "autogen-agentchat"

  Attempting uninstall: autogen-core
    Found existing installation: autogen-core 0.6.4
    Uninstalling autogen-core-0.6.4:
      Successfully uninstalled autogen-core-0.6.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [autogen-agentchat]
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install "autogen-ext[openai]"

Note: you may need to restart the kernel to use updated packages.


## Initializing Gemini LLM Model

In [7]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Defining the model_client with Gemini 1.5 Flash 8B
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=os.getenv("GOOGLE_API_KEY"),
)

response = await model_client.create([UserMessage(content="What is the capital of India?", source="user")])
print(response.content)

NotFoundError: Error code: 404 - [{'error': {'code': 404, 'message': 'models/gemini-1.5-flash-8b is not found for API version v1main, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}]

## Defining a Fake function call for Weather

In [8]:
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."

## Defining the Agent

In [34]:
from autogen_agentchat.agents import AssistantAgent # Importing Assistant agent 
from autogen_agentchat.ui import Console # Importing Console UI

weather_agent = AssistantAgent(
    model_client=model_client,
    name="weather_agent",
    tools=[get_weather],  # Registering the get_weather tool
    system_message="You are a helpful weather assistant. You can provide the current weather for any city using the get_weather tool.",
    reflect_on_tool_use=True,  # Enabling reflection on tool use
    model_client_stream=True,  # Enabling streaming responses
)

## Running the Agent

In [35]:
async def main() -> None: 
    # Running the agent in a console UI
    await Console(weather_agent.run_stream(task="What is the weather in San Francisco?"))
    await model_client.close()

await main()

---------- TextMessage (user) ----------
What is the weather in San Francisco?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='', arguments='{"city":"San Francisco"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in San Francisco is 73 degrees and Sunny.', name='get_weather', call_id='', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in San Francisco is 73 degrees and sunny.

